# User Activity Monitoring and Analytics
In this presentation, we will model a simple user activity model and use logical and visual tools to analyze this data

First, let's define some basic functions to prepare environment and generate random data

In [321]:
import random
import pandas as pd
from itertools import tee
import numpy as np

_pages = """
/Login
/Login/forgotpassword
/Logout
/Home
/Home/Events
/Home/TickerPriceHistory/
/Holdings
/Holdings/Details/%w
/Activities/
/Activities/Details/%w
/Alerts
/Elections/
/Elections/Details/%w
/AE/
/AE/Dashboard
/AE/RSU
/AE/RCA
/AE/SAR
/AE/ESPP
/Help
/Contacts
/PageNotFound
/ServerError
"""

dates_range = pd.date_range(start='1/4/2018 00:00:00', end='1/4/2018 23:59:59', freq='S')

pages = _pages.split('\n')[1:-1]

def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

def rand_userId():
    return random.randrange(500000,500200)

def rand_page():
    return random.choice(pages).replace('%w',str(random.randrange(1,10000)))

def rand_timestamp():
    return dates_range[random.randrange(0,dates_range.size-1)]

def rand_vectorized_timestamp(iterations):
    pos = 0
    _generated_range = []
    for i in range(iterations):
        pos = random.randrange(pos,dates_range.size-1)
        _generated_range.append(dates_range[pos])
    return _generated_range

def generate_single_user_activity(rows):
    id = rand_userId()
    _data = []
    _timestamps = rand_vectorized_timestamp(rows)
    for element in _timestamps:
        _data.append([id, element, rand_page()])
    return pd.DataFrame(data=_data, columns=['UserId','TimeStamp','URL'])

def generate_multiple_user_activity(rows):
    _data = []
    for i in range(rows):
        _data.append([rand_userId(),rand_timestamp(), rand_page()])
    return pd.DataFrame(data=_data, columns=['userid','timestamp','page'])

def calculate_timespent():
    for index, row in data.iterrows():
        try:
            next_row = data[(data.index > index) & (data.userid == row.userid)].iloc[0]
            delta = next_row.timestamp - row.timestamp
            delta_minutes = round(delta.total_seconds()/60)
            data.at[index, 'timespent'] = delta_minutes
        except IndexError as e:
            pass
        except Exception as ex:
            print(ex)
            
def show_links(userid):
    suv = data[data.userid==userid]
    print("%s records found for user %s" % (suv.shape[0], userid))
    for (i0, row0), (i1, row1) in pairwise(suv.iterrows()):
        print("%s (%s) -> %s" % (row0.page, row0.timespent, row1.page))
        
        #data.page.value_counts().head(50).plot(kind='bar',figsize=(20,20),x='page')

## Let's generate some random data

In [237]:
data = generate_multiple_user_activity(5000).sort_values('timestamp')

### Now that we have data, let's look at it

In [298]:
data.head(5)

,userid,timestamp,page,timespent
0,500181,2018-01-04 00:00:38,/Home,None
1,500161,2018-01-04 00:01:03,/AE/ESPP,None
2,500108,2018-01-04 00:01:12,/AE/RSU,None
3,500141,2018-01-04 00:01:24,/Home/TickerPriceHistory/,None
4,500151,2018-01-04 00:01:47,/Login,None


## Let's massage out data a little bit - sort by timestamp and align indexes
We will also add an extra column `timespent` to calculate how long users have spent of specific pages

In [302]:
data = data.reset_index(drop=True)
data['timespent'] = np.nan
pd.to_numeric(data.timespent,errors='raise')
data.head(5)

,userid,timestamp,page,timespent
0,500181,2018-01-04 00:00:38,/Home,NaN
1,500161,2018-01-04 00:01:03,/AE/ESPP,NaN
2,500108,2018-01-04 00:01:12,/AE/RSU,NaN
3,500141,2018-01-04 00:01:24,/Home/TickerPriceHistory/,NaN
4,500151,2018-01-04 00:01:47,/Login,NaN


#### We are going to calculate timespent on each page - it's not a very fast process, should take about a second per 1,000 records, but it's easily optimizable

In [303]:
%time calculate_timespent()

CPU times: user 7.95 s, sys: 0 ns, total: 7.95 s
Wall time: 7.97 s


### Let's look at the data now with time spent (in minutes)

In [304]:
data.head(5)

,userid,timestamp,page,timespent
0,500181,2018-01-04 00:00:38,/Home,49.0
1,500161,2018-01-04 00:01:03,/AE/ESPP,340.0
2,500108,2018-01-04 00:01:12,/AE/RSU,2.0
3,500141,2018-01-04 00:01:24,/Home/TickerPriceHistory/,10.0
4,500151,2018-01-04 00:01:47,/Login,81.0


In [305]:
data[data.userid==500181].head(5)

,userid,timestamp,page,timespent
0,500181,2018-01-04 00:00:38,/Home,49.0
171,500181,2018-01-04 00:50:06,/Elections/Details/3280,78.0
427,500181,2018-01-04 02:08:29,/AE/ESPP,6.0
455,500181,2018-01-04 02:14:41,/Alerts,57.0
648,500181,2018-01-04 03:11:30,/AE/ESPP,24.0


## Let's calculate some stats

Some hints: `data.describe()` also via `data.set_index([...]).stack.groupby(level=0).describe().unstack()`

In [306]:
print("%s unique visitors" % data.userid.unique().size)

200 unique visitors


## Time spent on pages statistics, per page, top {rows} results

In [376]:
rows=10
metrics = dict(size="Hits",sum="Sum",max="Max",min="Min",mean="Mean")
data.groupby('page').timespent.agg(metrics.keys()).rename(columns=metrics).head(rows)

,Hits,Sum,Max,Min,Mean
page,,,,,
/AE/,235,12789.0,337.0,0.0,55.847162
/AE/Dashboard,249,12518.0,304.0,0.0,51.941909
/AE/ESPP,248,12763.0,340.0,0.0,54.542735
/AE/RCA,213,11044.0,365.0,0.0,54.945274
/AE/RSU,236,13318.0,283.0,0.0,60.262443
/AE/SAR,258,12694.0,378.0,0.0,51.601626
/Activities/,220,11505.0,362.0,0.0,54.526066
/Activities/Details/1039,1,3.0,3.0,3.0,3.000000
/Activities/Details/1100,1,79.0,79.0,79.0,79.000000


## User-bases statistics

In [378]:
data.pivot_table(data, )

userid
500000             /AE/
500001    /AE/Dashboard
500002    /AE/Dashboard
500003             /AE/
500004             /AE/
500005    /AE/Dashboard
500006             /AE/
500007             /AE/
500008         /AE/ESPP
500009             /AE/
500010             /AE/
500011             /AE/
500012             /AE/
500013    /AE/Dashboard
500014             /AE/
500015             /AE/
500016             /AE/
500017          /AE/RCA
500018    /AE/Dashboard
500019    /AE/Dashboard
500020    /AE/Dashboard
500021             /AE/
500022         /AE/ESPP
500023    /AE/Dashboard
500024             /AE/
500025    /AE/Dashboard
500026    /AE/Dashboard
500027             /AE/
500028             /AE/
500029             /AE/
              ...      
500170             /AE/
500171             /AE/
500172          /AE/RCA
500173             /AE/
500174    /AE/Dashboard
500175             /AE/
500176             /AE/
500177             /AE/
500178             /AE/
500179             /AE/
500180   